In [1]:
from rosemary import jpt_parse_args, jpt_setup, jpt_in_notebook; jpt_setup()

if jpt_in_notebook():
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['CUDA_VISIBLE_DEVICES'].split(',')[0] 
    # '0,1,2,3,4,5'
    print(os.environ['CUDA_VISIBLE_DEVICES'])
# !nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


4


In [2]:
import re
import argparse
import json
import pickle
import string
import time
import matplotlib.pyplot as plt

import numpy as np
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sklearn.cluster import KMeans, AgglomerativeClustering
import torch
from tqdm import tqdm
import pandas as pd

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel

from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F

from note_pruning_analysis import get_lm_output, get_dataset, convert_example_to_str

In [3]:
model_name = 'all-mpnet-base-v2'
model_name_or_path = 'results/baselines/sentence-transformers/all-mpnet-base-v2'

dataset = 'wizardlm'
input_file = 'data/processed/sharegpt/sharegpt_data.jsonl'
input_file = 'data/processed/ultrachat/ultrachat_data.jsonl'
input_file = 'data/processed/wizardlm/wizardlm_data.jsonl'
n_clusters = 50
batch_size = 128

encode_fn_type = 'input'

In [4]:

model = AutoModel.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


[2023-11-13 21:38:06,314] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [5]:
ds = get_dataset(dataset, processed=True)

if encode_fn_type == 'input':
    def get_user_prompt_fn(example):
        example['text'] = example['messages'][0]['content']
        return example
    ds = ds.map(get_user_prompt_fn, num_proc=16)
ds

Found cached dataset json (/gpfs/u/home/PTFM/PTFMqngp/scratch/github/mitibm2023/external/open-instruct/data/processed/wizardlm/json/default-a3f71ae376e517fb/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


Dataset({
    features: ['dataset', 'id', 'messages'],
    num_rows: 143000
})

In [56]:
d = get_lm_output(dataset, 
                  model_name, 
                  encode_fn_type=encode_fn_type,
                  return_text_embedding=True,)
X = d['text_embedding']
X = X / np.linalg.norm(X, axis=-1, keepdims=True)

In [57]:
first_N = 10_000
ds = ds.select(range(first_N))
X = X[:first_N]

In [46]:
import time

from note_pruning_analysis import (
    clustering_run,
    sort_cluster_results_by_cluster_size,
    clustering_algorithm_scores,
    clustering_dist_to_centroids,
)

sort_by = f'cl=kmeans_nc={n_clusters}'
# sort_by = f'cl=kmeansminibatch_nc={n_clusters}'
# sort_by = f'cl=kmeansminibatch_nc={n_clusters}_bsz=256'


info = {}
info['N'] = len(X)
info['n_clusters'] = n_clusters


t0 = time.time()
Y, C, clustering_model = clustering_run(sort_by, X)
info['time_elapsed'] = time.time()-t0
info['inertia'] = clustering_model.inertia_

Y, C = sort_cluster_results_by_cluster_size(Y, C)
info.update(clustering_algorithm_scores(X, Y))


Initialization complete
Iteration 0, inertia 13046.0478515625.
Iteration 1, inertia 7963.150390625.
Iteration 2, inertia 7823.3193359375.
Iteration 3, inertia 7766.98486328125.
Iteration 4, inertia 7734.853515625.
Iteration 5, inertia 7714.99462890625.
Iteration 6, inertia 7701.08349609375.
Iteration 7, inertia 7689.7392578125.
Iteration 8, inertia 7681.154296875.
Iteration 9, inertia 7675.09619140625.
Iteration 10, inertia 7670.427734375.
Iteration 11, inertia 7666.078125.
Iteration 12, inertia 7661.98974609375.
Iteration 13, inertia 7658.16064453125.
Iteration 14, inertia 7654.4638671875.
Iteration 15, inertia 7651.60546875.
Iteration 16, inertia 7649.08056640625.
Iteration 17, inertia 7646.89208984375.
Iteration 18, inertia 7645.1513671875.
Iteration 19, inertia 7643.6064453125.
Iteration 20, inertia 7642.0751953125.
Iteration 21, inertia 7640.2158203125.
Iteration 22, inertia 7638.42626953125.
Iteration 23, inertia 7636.91845703125.
Iteration 24, inertia 7635.8076171875.
Iteration 

/gpfs/u/scratch/PTFM/PTFMqngp/miniconda3/envs/open-instruct/lib/python3.10/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [59]:
## what do I want to see 
#
# - unsupervised scores for the clustering algorithm
# - topic of each cluster
# - top-k closest text/distance for each cluster.
# - 2d tsne of clustering algorithm
#
# 

data = {
    'data_ind': np.arange(len(Y)),
    'cluster_assignment': Y,
    'dist_l2': clustering_dist_to_centroids(X, Y, C, device='cuda', dist='l2'),
    'dist_cd': clustering_dist_to_centroids(X, Y, C, device='cuda', dist='cd'),
}
df = pd.DataFrame(data)

import scipy
info['dist_l2_and_cd_spearmanr'] = scipy.stats.spearmanr(
    df['dist_l2'].to_numpy(), df['dist_cd'].to_numpy()).statistic

df

,data_ind,cluster_assignment,dist_l2,dist_cd
0,0,25,0.848907,0.465920
1,1,24,0.903208,0.562518
2,2,13,0.812550,0.407720
3,3,1,0.852474,0.476840
4,4,25,0.848171,0.464556
...,...,...,...,...
9995,9995,36,0.966166,0.707237
9996,9996,9,0.918524,0.604532
9997,9997,7,0.840771,0.457124
9998,9998,30,0.732729,0.318054


In [66]:

topk = 100

df_topk = df.sort_values(by=['cluster_assignment', 'dist_l2']) \
            .groupby('cluster_assignment') \
            .head(topk)
df_rand = df.sort_values(by=['cluster_assignment']) \
            .groupby('cluster_assignment') \
            .head(topk)

df_topk



,data_ind,cluster_assignment,dist_l2,dist_cd
5145,5145,0,0.922487,0.531328
6631,6631,0,0.923893,0.537723
4184,4184,0,0.930862,0.569574
1144,1144,0,0.935403,0.590459
5748,5748,0,0.939421,0.609022
...,...,...,...,...
4076,4076,49,0.905115,0.540195
715,715,49,0.905965,0.541407
6388,6388,49,0.923375,0.566490
810,810,49,0.949185,0.604552


In [103]:
def chat_completion_openai(
    messages,
    model='gpt-3.5-turbo-1106',
    temperature=0,
    max_tokens=256,
    ):

    import openai
    response = {}
    for _ in range(16):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                n=1,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            break
        except openai.error.OpenAIError as e:
            print(type(e), e)
            time.sleep(10)

    return response


In [171]:
i = 0



prompts = []
inds = df_topk[df_topk.cluster_assignment==i].iloc[:10].data_ind.to_list()
prompts += [ds[ind]['text'] for ind in inds]
inds = df_rand[df_rand.cluster_assignment==i].iloc[:10].data_ind.to_list()
prompts += [ds[ind]['text'] for ind in inds]
np.random.shuffle(prompts)


prompt = ("Given a list of user messages, use a few phrases to summarize a central topic "
          "for all messages in the list in English. Your output should only include a single line. "
          "Try to be specific. The topic should encompass "
         )
prompt = ("Given a list of user messages, output 8 words to summarize just one central topic of the instructions or queries. "
          "Be concise and specific. Avoid repeating phrases with similar meanings. "
          "Your output should fit in a single line.")
print(prompt)
prompt = prompt + "\n\n\nBEGIN OF THE MESSAGE LIST\n\n" + \
    '\n\n'.join([x.strip() for x in prompts]) + \
    "\n\nEND OF THE MESSAGE LIST."


messages = [
    {'role': 'user', 'content': prompt},
]


Given a list of user messages, output 8 words to summarize a SINGLE central topic of the instructions or queries. Be concise and specific. Avoid repeating phrases with similar meanings. Your output should fit in a single line.


In [172]:
model = 'gpt-3.5-turbo-1106'
# model = 'gpt-4'
response = chat_completion_openai(messages, model=model, max_tokens=256)
print(response['choices'][0]['message']['content'])

1. Creating a bash script to move modified files.
2. Impact of forcing function on control system stability.
3. Targeted delivery of therapeutic genes to brainstem.
4. Improving UAV detection and tracking with OAO and TDOA.
5. Incentivizing volunteers with C# code in non-profit org.
6. Best practices for code refactoring in C#.
7. Integration of Typescript to prevent type violations.
8. Incorporating audio features and IP rating using Go.


In [155]:
response['usage']['prompt_tokens']/1000*0.001 + response['usage']['completion_tokens']/1000*0.002

0.0017730000000000003

In [11]:
from fastchat_topic_clustering import get_topk_indices, print_topk

texts = [x[0]['content'] for x in ds[:len(labels)]['messages']]

k = 200
show_cut_off = 500
topk_indices = get_topk_indices(centers, labels, embeddings, k)
topk_str = print_topk(texts, labels, topk_indices, show_cut_off)
num_clusters = len(centers)

print(topk_indices.shape, len(topk_str))

torch.Size([50, 86]) 1747637


In [12]:
from collections import Counter
c = dict(Counter(labels.tolist()))
c = sorted(c.items())
c

# kmeans
# [(0, 142),
#  (1, 136),
#  (2, 116),
#  (3, 112),
#  (4, 109),
#  (5, 89),
#  (6, 82),
#  (7, 73),
#  (8, 73),
#  (9, 68)]

[(0, 476),
 (1, 310),
 (2, 307),
 (3, 295),
 (4, 276),
 (5, 274),
 (6, 265),
 (7, 262),
 (8, 250),
 (9, 245),
 (10, 239),
 (11, 228),
 (12, 227),
 (13, 221),
 (14, 218),
 (15, 217),
 (16, 217),
 (17, 217),
 (18, 214),
 (19, 211),
 (20, 211),
 (21, 204),
 (22, 204),
 (23, 203),
 (24, 201),
 (25, 200),
 (26, 194),
 (27, 188),
 (28, 181),
 (29, 180),
 (30, 177),
 (31, 176),
 (32, 168),
 (33, 168),
 (34, 168),
 (35, 167),
 (36, 158),
 (37, 155),
 (38, 149),
 (39, 149),
 (40, 146),
 (41, 140),
 (42, 137),
 (43, 137),
 (44, 137),
 (45, 125),
 (46, 124),
 (47, 101),
 (48, 97),
 (49, 86)]

In [14]:
from fastchat_topic_clustering import get_cluster_info

cluster_alg = 'kmeans'
# Dump results
filename_prefix = f"results/clustering/c{n_clusters}_{cluster_alg}"
os.makedirs(os.path.dirname(filename_prefix), exist_ok=True)

texts_np = np.array(texts)

with open(filename_prefix + "_topk.txt", "w") as fout:
    fout.write(topk_str)

with open(filename_prefix + "_all.jsonl", "w") as fout:
    for i in range(len(centers)):
        tmp_indices = labels == i
        tmp_embeddings = embeddings[tmp_indices]
        tmp_texts = texts_np[tmp_indices]

        scores = cos_sim(centers[i].unsqueeze(0), tmp_embeddings)[0]
        sorted_indices = torch.flip(torch.argsort(scores), dims=[0])

        for text, score in zip(tmp_texts[sorted_indices], scores[sorted_indices]):
            obj = {"cluster": i, "text": text, "sim": score.item()}
            fout.write(json.dumps(obj, ensure_ascii=False) + "\n")

# cluster_info = get_cluster_info(texts, labels, topk_indices)
# with open(filename_prefix + "_cluster.json", "w") as fout:
#     json.dump(cluster_info, fout, ensure_ascii=False)

In [ ]:
cluster_repr_examples = []






In [17]:

cluster_info = []
for k in range(len(topk_indices)):
    num_samples = torch.sum(labels == k).item()
    topk_prompts = []
    for idx in topk_indices[k]:
        topk_prompts.append(texts[idx])
    random_prompts = []
    for idx in range(len(topk_indices)):
        random_prompts.append(np.random.choice(texts))
    cluster_info.append((num_samples, topk_prompts, random_prompts))
    break
    


KeyboardInterrupt



In [33]:
# np.random.choice(text, size=100, replace=False)


inds = np.random.choice(len(text), size=len(topk_indices), replace=False)
random_prompts = [texts[i] for i in inds]


torch.Size([50, 86])

"Create an algorithm that identifies and categorizes each error message generated by a system and suggests potential solutions to resolve the issue. The algorithm should also provide a severity rating for each error based on its potential impact on the system's functionality."

In [38]:

# num_samples, len(topk_prompts)


topk_indices.shape



torch.Size([50, 86])

In [12]:

i = 12


t = dsf[i]['messages'][0]['content']
print(convert_example_to_str(0, dsf[i]))
print(len(t))
print(t)

metadata: {
    "dataset": "ultrachat",
    "id": "ultrachat_113",
    "idx": 0,
    "n_turns": 8
}

= USER        =
Here is a piece of text: DUBAI, Aug 26 (Reuters) - Saudi Arabia’s stock market outperformed by a large margin on Sunday, led by petrochemical stocks, as Middle Eastern bourses reopened after a one-week break for the Muslim festival of Eid Al Adha.
The Saudi stock index rose 1.4 percent, although trading volume was the lowest this year as some individual investors had not yet returned from holidays. Gainers outnumbered losers by 146 to 23.
The market has been buoyed this year by inflows of foreign funds in anticipation of Riyadh joining emerging market indexes next year. Recent exchange data indicates those inflows have slowed but not halted as stock valuations have risen.
Petrochemical makers were particularly strong after the Brent oil price rebounded above $75.0 a barrel last week. The top petrochemical firm, Saudi Basic Industries , added 1.8 percent and Saudi Kayan r

NameError: name 't' is not defined

In [30]:
print(under_min_length, above_max_length)

664 32991


In [17]:
from fastchat_topic_clustering import get_embeddings


embeddings = get_embeddings(texts, args.model, args.batch_size)

[2023-11-08 00:02:46,673] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Batches:   4%|▍         | 53/1202 [01:36<34:53,  1.82s/it]

KeyboardInterrupt



Error in callback <bound method AutoreloadMagics.post_execute_hook of <IPython.extensions.autoreload.AutoreloadMagics object at 0x7f19c4a4e590>> (for post_execute):


KeyboardInterrupt: 

In [16]:
from fastchat_topic_clustering import get_embeddings


embeddings = get_embeddings(texts, args.model, args.batch_size)
if args.cluster_alg == "kmeans":
    centers, labels = run_k_means(embeddings, num_clusters)
elif args.cluster_alg == "aggcls":
    centers, labels = run_agg_cluster(embeddings, num_clusters)
elif args.cluster_alg == "HDBSCAN":
    centers, labels = run_hdbscan_cluster(embeddings)
else:
    raise ValueError(f"Invalid clustering algorithm: {args.cluster_alg}")


307563